In [253]:
import urllib3
from urllib3.exceptions import InsecureRequestWarning

import os
import certifi
import pandas as pd
import requests

from functools import reduce

In [254]:
# 경고 끄기
urllib3.disable_warnings(InsecureRequestWarning)   

startYYYYMM = '201801'
endYYYYMM = '202212'

startYYYYQQ = '201801'
endYYYYQQ = '202212'

startYYYY = '2018'
endYYYY = '2022'

# 실행 시 kosisAPI 개인 인증키 사용하실 것
apiKey = 'NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM'

In [255]:
# URL request 함수
def urlRequest(url):
    r = requests.get(url, verify=False)
    jo = r.json()
    return  pd.DataFrame(jo)

In [256]:
# initQuaterDf 함수
# 데이터마다 시계열이 다르므로, 공통 시계열(2019Q1 ~ 2022Q4) DataFrame을 만들어서 추출된 데이터를 붙임(Left Join) 
# 연도 및 분기 리스트
def initQuaterDf():
    years = range(2018, 2023)  # 2018 ~ 2022
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']
    # PRD_DE 값 생성
    prd_de_list = [f"{year}{q}" for year in years for q in quarters]
    # DataFrame 생성 (열 이름이 'PRD_DE')
    return pd.DataFrame({'PRD_DE': prd_de_list})

In [257]:
# 추출 데이터가 월단위 데이터이므로, 이를 Quater 단위로 변환
# 월 → 분기(Q1~Q4)로 변환하는 함수 : 3,6,9,12월 자료만 Quater로 변환하고 나머지는 Unknown 처리
def month_to_quarter(prd):
    year = prd[:4]
    month = prd[-2:]
    if month in ['03']:
        quarter = 'Q1'
    elif month in ['06']:
        quarter = 'Q2'
    elif month in ['09']:
        quarter = 'Q3'
    elif month in ['12']:
        quarter = 'Q4'
    else:
        quarter = 'Unknown'
    return f"{year}{quarter}"

In [258]:
def save_excel_and_csv(df, filename):
    folder_name = 'output_files'
    # 1. 폴더가 없으면 생성
    os.makedirs(folder_name, exist_ok=True)
    # 2. 저장할 전체 경로 만들기
    excel_path = os.path.join(folder_name, f"{filename}.xlsx")
    csv_path = os.path.join(folder_name, f"{filename}.csv")
     # 3. 파일 저장
    df.to_excel(excel_path, index=False)
    df.to_csv(csv_path, index=False, encoding='utf-8')

In [259]:
# 총인구수(전국/시도/시/군/구)(단위: 명)
# 총가구수(전국/시도/시/군/구)(단위: 가구)
# 총주택수(전국/시도/시/군/구)

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy() 
    newColName = tempDf['ITM_NM'].unique()[0]          # 'DT' 컬럼 네임 교체
    tempDf.rename(columns={'DT': newColName}, inplace=True)
    return tempDf[['PRD_DE', 'C1_NM', newColName]]

#################### 총인구수(전국/시도/시/군/구)(단위: 명) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T00+&objL1=ALL&objL2=ALL&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=INH_1IN1503_01'
populationDf = dataRefine(urlRequest(url))
################################################################################

##################### 총가구수(전국/시도/시/군/구)(단위: 가구) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T1100+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=INH_1JC1501'
houseHouldDf = dataRefine(urlRequest(url))
##################################################################################

##################### 총주택수(전국/시도/시/군/구) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T10+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=INH_1JU1501'
houseDf = dataRefine(urlRequest(url))
#######################################################################

dfs = [populationDf, houseHouldDf, houseDf]
# 모든 df를 PRD_DE, C1_NM 기준으로 내부 조인
popStatDf = reduce(lambda left, right: pd.merge(left, right, on=['PRD_DE', 'C1_NM']), dfs)
display(popStatDf)
save_excel_and_csv(popStatDf, 'popStatDf')

,PRD_DE,C1_NM,총인구(명),총가구_가구,주택
0,2018,전국,51629512,20499543,17633327
1,2019,전국,51779203,20891348,18126954
2,2020,전국,51829136,21484785,18525844
3,2021,전국,51738071,22022753,18811627
4,2022,전국,51692272,22383187,19155585
...,...,...,...,...,...
5350,2018,서귀포시,175350,70383,67848
5351,2019,서귀포시,177360,71461,71357
5352,2020,서귀포시,178552,73627,72684
5353,2021,서귀포시,179238,76208,74077


In [260]:
# 소상공인 경기동향(BSI) 지수 (소상공인, 부문별 실적 및 전망)

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = tempDf['ITM_NM'].drop_duplicates().values.tolist()
    filter_2 = tempDf['C1_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        for p in filter_2:
            # 'DT'컬럼의 이름을 대체함
            newColName = f"{tempDf['TBL_NM'].unique()[0][:]}_{p}_{s}"  
            filtered = tempDf[(tempDf['ITM_NM'] == s) & (tempDf['C1_NM'] == p)]
            split_dfs[newColName] = filtered.copy()
            split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
            split_dfs[newColName]['PRD_DE'] = split_dfs[newColName]['PRD_DE'].apply(month_to_quarter)
            # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
            split_dfs[newColName] = split_dfs[newColName][~split_dfs[newColName]['PRD_DE'].str.contains('Unknown')]
    return split_dfs
 
def mergeTimeSeriesDataFrame(split_dfs):
    df = initQuaterDf()
    df_merged = df.copy()
    for name, sub in split_dfs.items():
        sub_unique = sub.drop_duplicates(subset=['PRD_DE'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 4]], on='PRD_DE', how='left', suffixes=('', '_dup'))  # 필요에 따라 'outer' 등으로 변경
    return df_merged
    

##################### 소상공인 경기동향(BSI) 지수 (소상공인, 부문별 실적 및 전망) #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_001'
msBsiDf = mergeTimeSeriesDataFrame(dataRefine(urlRequest(url)))
display(msBsiDf)
save_excel_and_csv(msBsiDf, 'msBsiDf')
#########################################################################################################

,PRD_DE,소상공인 부문별 실적 및 전망_경기전반_체감,소상공인 부문별 실적 및 전망_매출_체감,소상공인 부문별 실적 및 전망_영업이익_체감,소상공인 부문별 실적 및 전망_자금사정_체감,소상공인 부문별 실적 및 전망_원재료조달사정_체감,소상공인 부문별 실적 및 전망_재고_체감,소상공인 부문별 실적 및 전망_고용_체감,소상공인 부문별 실적 및 전망_비용상황_체감,소상공인 부문별 실적 및 전망_구매고객수_체감,소상공인 부문별 실적 및 전망_경기전반_전망,소상공인 부문별 실적 및 전망_매출_전망,소상공인 부문별 실적 및 전망_영업이익_전망,소상공인 부문별 실적 및 전망_자금사정_전망,소상공인 부문별 실적 및 전망_원재료조달사정_전망,소상공인 부문별 실적 및 전망_재고_전망,소상공인 부문별 실적 및 전망_고용_전망,소상공인 부문별 실적 및 전망_비용상황_전망,소상공인 부문별 실적 및 전망_구매고객수_전망
0,2018Q1,79.7,80.1,79.9,80,94,NaN,NaN,NaN,NaN,103,102.4,102.1,100.6,102.9,NaN,NaN,NaN,NaN
1,2018Q2,62.2,61.7,61.7,63.3,90.8,NaN,NaN,NaN,NaN,84.7,84.3,84.2,84.8,97.2,NaN,NaN,NaN,NaN
2,2018Q3,67.9,68.2,67.5,69.7,87.8,NaN,NaN,NaN,NaN,99.4,99.1,99,98.7,103.6,NaN,NaN,NaN,NaN
3,2018Q4,62.1,61.9,61.9,63.6,95.6,NaN,NaN,NaN,NaN,86.8,87.1,86.8,86.8,97.2,NaN,NaN,NaN,NaN
4,2019Q1,73.3,73.3,73.2,73.6,93.3,NaN,NaN,NaN,NaN,96,95.6,95.5,95.1,103.7,NaN,NaN,NaN,NaN
5,2019Q2,65.6,65.8,65.7,65.8,89.5,NaN,NaN,NaN,NaN,82.1,81.9,81.6,81.9,97.4,NaN,NaN,NaN,NaN
6,2019Q3,76.4,76.5,76.5,76.9,98.7,NaN,NaN,NaN,NaN,92,91.9,91.9,90.2,97,NaN,NaN,NaN,NaN
7,2019Q4,66.4,64.8,64.8,66.8,94.3,NaN,NaN,NaN,NaN,85.8,86.1,85.9,86,97.6,NaN,NaN,NaN,NaN
8,2020Q1,29.7,29.3,NaN,31.3,NaN,105.8,98.4,NaN,NaN,94.1,94.3,NaN,93.7,NaN,99.9,99.4,NaN,NaN
9,2020Q2,82.6,82.4,NaN,81.8,NaN,101.3,99.8,NaN,NaN,98.9,99,NaN,98.3,NaN,100.7,100,NaN,NaN


In [261]:
# 소상공인 경기동향(BSI) 지수 (소상공인, 업종별 실적 및 전망)
# 소상공인 경기동향(BSI) 지수 (소상공인, 지역별 실적 및 전망)

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = temp_df['ITM_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        # 'DT'컬럼의 이름을 대체함
        newColName = f"{temp_df['TBL_NM'].unique()[0][:]}_{s}"        
        filtered = temp_df[temp_df['ITM_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
        split_dfs[newColName]['PRD_DE'] = split_dfs[newColName]['PRD_DE'].apply(month_to_quarter)
        # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
        split_dfs[newColName] = split_dfs[newColName][~split_dfs[newColName]['PRD_DE'].str.contains('Unknown')]
    return split_dfs

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df[['PRD_DE', 'C1_NM']].copy()
    for name, sub in split_dfs.items():
        df_merged = df_merged.merge(sub.iloc[:, [2, 3, 4]], on=['PRD_DE', 'C1_NM'], how='left')  
    return df_merged
    
def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return df.merge(refDf, on='PRD_DE', how='left')  


#################### 소상공인 경기동향(BSI) 지수 (소상공인, 업종별 실적 및 전망) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_002'
msInduBsiDf = mergeTimeSeriesDataFrame(mergeDataFrame(dataRefine(urlRequest(url))))
display(msInduBsiDf)
save_excel_and_csv(msInduBsiDf, 'msInduBsiDf')
####################################################################################################


##################### 소상공인 경기동향(BSI) 지수 (소상공인, 지역별 실적 및 전망) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_005'
msLocalBsiDf = mergeTimeSeriesDataFrame(mergeDataFrame( dataRefine(urlRequest(url))))
display(msLocalBsiDf)
save_excel_and_csv(msInduBsiDf, 'msInduBsiDf')
####################################################################################################

,PRD_DE,C1_NM,소상공인 부문별 실적 및 전망_경기전반_체감,소상공인 부문별 실적 및 전망_매출_체감,소상공인 부문별 실적 및 전망_영업이익_체감,소상공인 부문별 실적 및 전망_자금사정_체감,소상공인 부문별 실적 및 전망_원재료조달사정_체감,소상공인 부문별 실적 및 전망_재고_체감,소상공인 부문별 실적 및 전망_고용_체감,소상공인 부문별 실적 및 전망_비용상황_체감,소상공인 부문별 실적 및 전망_구매고객수_체감,소상공인 부문별 실적 및 전망_경기전반_전망,소상공인 부문별 실적 및 전망_매출_전망,소상공인 부문별 실적 및 전망_영업이익_전망,소상공인 부문별 실적 및 전망_자금사정_전망,소상공인 부문별 실적 및 전망_원재료조달사정_전망,소상공인 부문별 실적 및 전망_재고_전망,소상공인 부문별 실적 및 전망_고용_전망,소상공인 부문별 실적 및 전망_비용상황_전망,소상공인 부문별 실적 및 전망_구매고객수_전망
0,2018Q1,소매업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018Q1,음식점업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018Q1,부동산업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018Q1,전문과학 기술사업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018Q1,교육 서비스업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2022Q4,교육 서비스업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,2022Q4,스포츠 및 오락관련,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,2022Q4,수리업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,2022Q4,개인 서비스업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,PRD_DE,C1_NM,소상공인 부문별 실적 및 전망_경기전반_체감,소상공인 부문별 실적 및 전망_매출_체감,소상공인 부문별 실적 및 전망_영업이익_체감,소상공인 부문별 실적 및 전망_자금사정_체감,소상공인 부문별 실적 및 전망_원재료조달사정_체감,소상공인 부문별 실적 및 전망_재고_체감,소상공인 부문별 실적 및 전망_고용_체감,소상공인 부문별 실적 및 전망_비용상황_체감,소상공인 부문별 실적 및 전망_구매고객수_체감,소상공인 부문별 실적 및 전망_경기전반_전망,소상공인 부문별 실적 및 전망_매출_전망,소상공인 부문별 실적 및 전망_영업이익_전망,소상공인 부문별 실적 및 전망_자금사정_전망,소상공인 부문별 실적 및 전망_원재료조달사정_전망,소상공인 부문별 실적 및 전망_재고_전망,소상공인 부문별 실적 및 전망_고용_전망,소상공인 부문별 실적 및 전망_비용상황_전망,소상공인 부문별 실적 및 전망_구매고객수_전망
0,2018Q1,서울,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018Q1,부산,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018Q1,대구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018Q1,인천,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018Q1,광주,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,2022Q4,전북,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,2022Q4,전남,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,2022Q4,경북,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338,2022Q4,경남,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [262]:
# 소상공인 경기동향(BSI) 지수 (전통시장, 부문별 실적 및 전망)

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = tempDf['ITM_NM'].drop_duplicates().values.tolist()
    filter_2 = tempDf['C1_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        for p in filter_2:
            # 'DT'컬럼의 이름을 대체함
            newColName = f"{tempDf['TBL_NM'].unique()[0][:]}_{p}_{s}"  
            filtered = tempDf[(tempDf['ITM_NM'] == s) & (tempDf['C1_NM'] == p)]
            split_dfs[newColName] = filtered.copy()
            split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
            split_dfs[newColName]['PRD_DE'] = split_dfs[newColName]['PRD_DE'].apply(month_to_quarter)
            # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
            split_dfs[newColName] = split_dfs[newColName][~split_dfs[newColName]['PRD_DE'].str.contains('Unknown')]
    return split_dfs
 
def mergeTimeSeriesDataFrame(split_dfs):
    df = initQuaterDf()
    df_merged = df.copy()
    for name, sub in split_dfs.items():
        sub_unique = sub.drop_duplicates(subset=['PRD_DE'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 4]], on='PRD_DE', how='left', suffixes=('', '_dup'))  # 필요에 따라 'outer' 등으로 변경
    return df_merged
    

##################### 소상공인 경기동향(BSI) 지수 (전통시장, 부문별 실적 및 전망) #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_003'
traditionalMarketBsiDf = mergeTimeSeriesDataFrame(dataRefine(urlRequest(url)))
display(traditionalMarketBsiDf)
save_excel_and_csv(traditionalMarketBsiDf, 'traditionalMarketBsiDf')
####################################################################################################

,PRD_DE,전통시장 부문별 실적 및 전망_경기전반_체감,전통시장 부문별 실적 및 전망_매출_체감,전통시장 부문별 실적 및 전망_영업이익_체감,전통시장 부문별 실적 및 전망_자금사정_체감,전통시장 부문별 실적 및 전망_원재료조달사정_체감,전통시장 부문별 실적 및 전망_구매고객수_체감,전통시장 부문별 실적 및 전망_재고_체감,전통시장 부문별 실적 및 전망_고용_체감,전통시장 부문별 실적 및 전망_비용상황_체감,전통시장 부문별 실적 및 전망_경기전반_전망,전통시장 부문별 실적 및 전망_매출_전망,전통시장 부문별 실적 및 전망_영업이익_전망,전통시장 부문별 실적 및 전망_자금사정_전망,전통시장 부문별 실적 및 전망_원재료조달사정_전망,전통시장 부문별 실적 및 전망_구매고객수_전망,전통시장 부문별 실적 및 전망_재고_전망,전통시장 부문별 실적 및 전망_고용_전망,전통시장 부문별 실적 및 전망_비용상황_전망
0,2018Q1,67,65.8,65.7,63.6,107.5,65.7,NaN,NaN,NaN,92.6,91.3,91.2,90.2,97,91.1,NaN,NaN,NaN
1,2018Q2,54.1,53.2,53.1,54.1,82.2,53.9,NaN,NaN,NaN,76.8,76,75.1,76.7,91.7,74.9,NaN,NaN,NaN
2,2018Q3,73.2,73.2,73,73.8,89.6,75,NaN,NaN,NaN,106.8,107,106.7,105,105.2,108.1,NaN,NaN,NaN
3,2018Q4,52.7,52.7,52.9,54.9,95,53.5,NaN,NaN,NaN,82.8,83,82.9,83.1,102,82,NaN,NaN,NaN
4,2019Q1,62.4,62.1,62,64.4,88.2,65.2,NaN,NaN,NaN,91.3,91.5,91.3,89,97,91.2,NaN,NaN,NaN
5,2019Q2,57,56.9,56.8,57.3,80.5,57.6,NaN,NaN,NaN,80,80,79.9,79.8,100.4,79.5,NaN,NaN,NaN
6,2019Q3,84.6,84.2,84.3,84.3,99,86.4,NaN,NaN,NaN,102.3,102.8,102.6,98.9,101.8,103.1,NaN,NaN,NaN
7,2019Q4,65.4,65.6,65.7,66.6,100.3,66.3,NaN,NaN,NaN,85.3,86.1,85.9,86,101.2,85.7,NaN,NaN,NaN
8,2020Q1,28.4,28.3,NaN,30.1,NaN,NaN,12.7,99.6,NaN,95,95.2,NaN,94.9,NaN,NaN,100.7,98,NaN
9,2020Q2,79.2,78.2,NaN,80.3,NaN,NaN,103,99.5,NaN,103.2,103.4,NaN,103.4,NaN,NaN,100,100,NaN


In [263]:
# 소상공인 경기동향(BSI) 지수 (전통시장, 업종별 실적 및 전망)
# 소상공인 경기동향(BSI) 지수 (전통시장, 지역별 실적 및 전망)

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = temp_df['ITM_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        # 'DT'컬럼의 이름을 대체함
        newColName = f"{temp_df['TBL_NM'].unique()[0][:]}_{s}"        
        filtered = temp_df[temp_df['ITM_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
        split_dfs[newColName]['PRD_DE'] = split_dfs[newColName]['PRD_DE'].apply(month_to_quarter)
        # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
        split_dfs[newColName] = split_dfs[newColName][~split_dfs[newColName]['PRD_DE'].str.contains('Unknown')]
    return split_dfs

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df[['PRD_DE', 'C1_NM']].copy()
    for name, sub in split_dfs.items():
        sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4]], on=['PRD_DE', 'C1_NM'], how='left')  
    return df_merged
    
def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return df.merge(refDf, on='PRD_DE', how='left')  


##################### 소상공인 경기동향(BSI) 지수 (전통시장, 업종별 실적 및 전망) #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_004'
traditionalMarketInduBsiDf = mergeTimeSeriesDataFrame(mergeDataFrame(dataRefine(urlRequest(url))))
display(traditionalMarketInduBsiDf)
save_excel_and_csv(traditionalMarketInduBsiDf, 'traditionalMarketInduBsiDf')
#########################################################################################################


###################### 소상공인 경기동향(BSI) 지수 (전통시장, 지역별 실적 및 전망) #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_006'
traditionalMarketLocalBsiDf = mergeTimeSeriesDataFrame(mergeDataFrame(dataRefine(urlRequest(url))))
display(traditionalMarketLocalBsiDf)
save_excel_and_csv(traditionalMarketLocalBsiDf, 'traditionalMarketLocalBsiDf')
#########################################################################################################

,PRD_DE,C1_NM,소상공인 부문별 실적 및 전망_경기전반_체감,소상공인 부문별 실적 및 전망_매출_체감,소상공인 부문별 실적 및 전망_영업이익_체감,소상공인 부문별 실적 및 전망_자금사정_체감,소상공인 부문별 실적 및 전망_원재료조달사정_체감,소상공인 부문별 실적 및 전망_재고_체감,소상공인 부문별 실적 및 전망_고용_체감,소상공인 부문별 실적 및 전망_비용상황_체감,소상공인 부문별 실적 및 전망_구매고객수_체감,소상공인 부문별 실적 및 전망_경기전반_전망,소상공인 부문별 실적 및 전망_매출_전망,소상공인 부문별 실적 및 전망_영업이익_전망,소상공인 부문별 실적 및 전망_자금사정_전망,소상공인 부문별 실적 및 전망_원재료조달사정_전망,소상공인 부문별 실적 및 전망_재고_전망,소상공인 부문별 실적 및 전망_고용_전망,소상공인 부문별 실적 및 전망_비용상황_전망,소상공인 부문별 실적 및 전망_구매고객수_전망
0,2018Q1,농산물,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018Q1,축산물,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018Q1,수산물,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018Q1,가공식품,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018Q1,의류·신발,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2022Q4,가공식품,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,2022Q4,의류·신발,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,2022Q4,가정용품,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,2022Q4,음식점업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,PRD_DE,C1_NM,소상공인 부문별 실적 및 전망_경기전반_체감,소상공인 부문별 실적 및 전망_매출_체감,소상공인 부문별 실적 및 전망_영업이익_체감,소상공인 부문별 실적 및 전망_자금사정_체감,소상공인 부문별 실적 및 전망_원재료조달사정_체감,소상공인 부문별 실적 및 전망_재고_체감,소상공인 부문별 실적 및 전망_고용_체감,소상공인 부문별 실적 및 전망_비용상황_체감,소상공인 부문별 실적 및 전망_구매고객수_체감,소상공인 부문별 실적 및 전망_경기전반_전망,소상공인 부문별 실적 및 전망_매출_전망,소상공인 부문별 실적 및 전망_영업이익_전망,소상공인 부문별 실적 및 전망_자금사정_전망,소상공인 부문별 실적 및 전망_원재료조달사정_전망,소상공인 부문별 실적 및 전망_재고_전망,소상공인 부문별 실적 및 전망_고용_전망,소상공인 부문별 실적 및 전망_비용상황_전망,소상공인 부문별 실적 및 전망_구매고객수_전망
0,2018Q1,서울,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018Q1,부산,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018Q1,대구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018Q1,인천,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018Q1,광주,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,2022Q4,전북,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,2022Q4,전남,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,2022Q4,경북,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338,2022Q4,경남,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [264]:
# 종업원 규모별 설비투자(서비스업, 300명 미만 사업장)(단위: 억원)
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T1+&objL1=001001001+001001002+001001003+001001004+001001005+001001006+001001007+001001008+001001009+001001010+001001011+001001011001+001001011002+001001011003+001001011004+001001012+001001013+001001013001+001001013002+001001014+001001014001+001001014002+001001014003+001001015+001001015001+001001015002+001001015003+001001016+001001017+001001017001+001001017002+001001017003+001001017004+001001017005+001001018+001001019+001001019001+001001019002+001001019003+001001019004+001001020+001001020001+001001020002+001001021+001001021001+001001021002+001001022+001001022001+001001022002+001001023+001001024+001002+001002001+001002002+001002003+001002004+001002005+001002005001+001002005002+001002005003+001002006+001002006001+001002006002+001002006003+001002006004+001002007+001002007001+001002007002+001002008+001002008001+001002008002+001002008003+001002008004+001002008005+001002008006+001002009+001002010+001002011+&objL2=000+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=406&tblId=DT_406N_302001_A006'
df = urlRequest(url)
tempDf = df[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()

# 'DT' 컬럼 네임을 '종업원 규모별 설비투자'으로 교체
newColName = tempDf['ITM_NM'].unique()[0]
tempDf.rename(columns={'DT': newColName}, inplace=True)
capitalSpendDf = tempDf[['PRD_DE', 'C1_NM', newColName]]
display(capitalSpendDf)
save_excel_and_csv(capitalSpendDf, 'capitalSpendDf')

,PRD_DE,C1_NM,종업원 규모별 설비투자
0,2021,식료품,5089.32014941
1,2022,식료품,5137.251696
2,2021,음료,1321.73333333
3,2022,음료,1313.865833
4,2021,담배,0
...,...,...,...
151,2022,부동산업,770.419643
152,2021,전문 과학 및 기술 서비스업,4785.17119867
153,2022,전문 과학 및 기술 서비스업,1363.032876
154,2021,보건업 및 사회복지 서비스업,1100.82727273


In [265]:
# 경기전반 실적 SBHI
# 경기전반 전망 SBHI
# 내수판매 실적 SBHI
# 수출실적 SBHI
# 수출전망 SBHI
# 영업이익 실적 SBHI
# 영업이익 전망 SBHI
# 자금사정 실적 SBHI
# 자금사정 전망 SBHI
# 고용수준 실적 SBHI
# 고용수준 전망 SBHI

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # 'DT' 컬럼 네임을 'TBL_NM'으로 교체
    newColName = temp_df['TBL_NM'].unique()[0]
    temp_df.rename(columns={'DT': newColName}, inplace=True)
    temp_df[['PRD_DE', 'C1_NM', newColName]]
    # 월 → 분기(Q1~Q4)로 변환 : 3,6,9,12월 자료만 Quater로 변환하고 나머지는 Unknown 처리
    temp_df['PRD_DE'] = temp_df['PRD_DE'].apply(month_to_quarter)
    # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
    return temp_df[~temp_df['PRD_DE'].str.contains('Unknown')]

def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return  df.merge(refDf, on='PRD_DE', how='left') 


##################### 경기전반 실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10100'
refinedDf = dataRefine(urlRequest(url))
economicPerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(economicPerfDf)
###############################################################


###################### 경기전반 전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10101'
refinedDf = dataRefine(urlRequest(url))
economicPredfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(economicPredfDf)
###############################################################


###################### 내수판매 실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10104'
refinedDf = dataRefine(urlRequest(url))
domesticPerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(domesticPerfDf)
###############################################################


###################### 내수판매 전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10105'
refinedDf = dataRefine(urlRequest(url))
domesticPredDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(domesticPredDf)
###############################################################


###################### 수출실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10106'
refinedDf = dataRefine(urlRequest(url))
exportPerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(exportPerfDf)
###############################################################


###################### 수출전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10107'
refinedDf = dataRefine(urlRequest(url))
exportPredDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(exportPredDf)
###############################################################


###################### 영업이익 실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10108'
refinedDf = dataRefine(urlRequest(url))
opProfitPerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(opProfitPerfDf)
###############################################################


###################### 영업이익 전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10109'
refinedDf = dataRefine(urlRequest(url))
opProfitPredDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(opProfitPredDf)
###############################################################


###################### 자금사정 실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10110'
refinedDf = dataRefine(urlRequest(url))
fundPerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(fundPerfDf)
###############################################################


###################### 자금사정 전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10111'
refinedDf = dataRefine(urlRequest(url))
fundPredDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(fundPredDf)
###############################################################


###################### 고용수준 실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10118'
refinedDf = dataRefine(urlRequest(url))
employeePerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(employeePerfDf)
###############################################################


###################### 고용수준 전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10119'
refinedDf = dataRefine(urlRequest(url))
employeePredDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(employeePredDf)
###############################################################


dfs = [economicPerfDf, economicPredfDf, domesticPerfDf, domesticPredDf, exportPerfDf, exportPredDf, opProfitPerfDf, opProfitPredDf, fundPerfDf, fundPredDf, employeePerfDf, employeePredDf]
# 모든 df를 PRD_DE, C1_NM 기준으로 내부 조인
econoPerfPredDf = reduce(lambda left, right: pd.merge(left, right, on=['PRD_DE', 'C1_NM']), dfs)
display(econoPerfPredDf)
save_excel_and_csv(econoPerfPredDf, 'econoPerfPredDf')

,PRD_DE,C1_NM,경기전반 실적 SBHI
0,2018Q1,서비스업,87.7
1,2018Q1,도매 및 소매업,89.9
2,2018Q1,수리 및 기타개인서비스업,73.5
3,2018Q1,운수업,83.7
4,2018Q1,숙박 및 음식점업,73.3
...,...,...,...
215,2022Q4,부동산업 및 임대업,70.1
216,2022Q4,전문과학 및 기술서비스업,85.9
217,2022Q4,사업시설관리 및 사업지원서비스업,85.8
218,2022Q4,교육서비스업,76.7


,PRD_DE,C1_NM,경기전반 전망 SBHI
0,2018Q1,서비스업,89.2
1,2018Q1,도매 및 소매업,87.8
2,2018Q1,수리 및 기타개인서비스업,92.6
3,2018Q1,운수업,91.2
4,2018Q1,숙박 및 음식점업,89.3
...,...,...,...
215,2022Q4,부동산업 및 임대업,67.4
216,2022Q4,전문과학 및 기술서비스업,87.8
217,2022Q4,사업시설관리 및 사업지원서비스업,88.9
218,2022Q4,교육서비스업,87.2


,PRD_DE,C1_NM,내수판매 실적 SBHI
0,2018Q1,서비스업,87.9
1,2018Q1,도매 및 소매업,89.6
2,2018Q1,수리 및 기타개인서비스업,74.2
3,2018Q1,운수업,83.9
4,2018Q1,숙박 및 음식점업,78.7
...,...,...,...
215,2022Q4,부동산업 및 임대업,70.1
216,2022Q4,전문과학 및 기술서비스업,86.3
217,2022Q4,사업시설관리 및 사업지원서비스업,85.3
218,2022Q4,교육서비스업,76.6


,PRD_DE,C1_NM,내수판매 전망 SBHI
0,2018Q1,서비스업,88.9
1,2018Q1,도매 및 소매업,87.5
2,2018Q1,수리 및 기타개인서비스업,93.1
3,2018Q1,운수업,92.9
4,2018Q1,숙박 및 음식점업,87.9
...,...,...,...
215,2022Q4,부동산업 및 임대업,68.1
216,2022Q4,전문과학 및 기술서비스업,87.2
217,2022Q4,사업시설관리 및 사업지원서비스업,88
218,2022Q4,교육서비스업,86.5


,PRD_DE,C1_NM,수출실적 SBHI
0,2018Q1,서비스업,88
1,2018Q1,도매 및 소매업,88
2,2018Q1,수리 및 기타개인서비스업,73
3,2018Q1,운수업,111.8
4,2018Q1,숙박 및 음식점업,-
...,...,...,...
215,2022Q4,부동산업 및 임대업,72.9
216,2022Q4,전문과학 및 기술서비스업,75.7
217,2022Q4,사업시설관리 및 사업지원서비스업,97.2
218,2022Q4,교육서비스업,-


...

,PRD_DE,C1_NM,자금사정 실적 SBHI
0,2018Q1,서비스업,79
1,2018Q1,도매 및 소매업,78.3
2,2018Q1,수리 및 기타개인서비스업,67.1
3,2018Q1,운수업,79.1
4,2018Q1,숙박 및 음식점업,71.3
...,...,...,...
215,2022Q4,부동산업 및 임대업,70.9
216,2022Q4,전문과학 및 기술서비스업,84.9
217,2022Q4,사업시설관리 및 사업지원서비스업,90.2
218,2022Q4,교육서비스업,75.9


,PRD_DE,C1_NM,자금사정 전망 SBHI
0,2018Q1,서비스업,82.6
1,2018Q1,도매 및 소매업,81.8
2,2018Q1,수리 및 기타개인서비스업,82.1
3,2018Q1,운수업,87.1
4,2018Q1,숙박 및 음식점업,76.7
...,...,...,...
215,2022Q4,부동산업 및 임대업,68.1
216,2022Q4,전문과학 및 기술서비스업,86.4
217,2022Q4,사업시설관리 및 사업지원서비스업,85.2
218,2022Q4,교육서비스업,83.4


,PRD_DE,C1_NM,고용수준 실적 SBHI
0,2018Q1,서비스업,93.5
1,2018Q1,도매 및 소매업,92.4
2,2018Q1,수리 및 기타개인서비스업,93
3,2018Q1,운수업,89.3
4,2018Q1,숙박 및 음식점업,103.9
...,...,...,...
215,2022Q4,부동산업 및 임대업,100.2
216,2022Q4,전문과학 및 기술서비스업,94.4
217,2022Q4,사업시설관리 및 사업지원서비스업,83.9
218,2022Q4,교육서비스업,99.2


,PRD_DE,C1_NM,고용수준 전망 SBHI
0,2018Q1,서비스업,95.1
1,2018Q1,도매 및 소매업,95.3
2,2018Q1,수리 및 기타개인서비스업,90.9
3,2018Q1,운수업,89.8
4,2018Q1,숙박 및 음식점업,101.5
...,...,...,...
215,2022Q4,부동산업 및 임대업,94.6
216,2022Q4,전문과학 및 기술서비스업,93.3
217,2022Q4,사업시설관리 및 사업지원서비스업,83.8
218,2022Q4,교육서비스업,99.2


,PRD_DE,C1_NM,경기전반 실적 SBHI,경기전반 전망 SBHI,내수판매 실적 SBHI,내수판매 전망 SBHI,수출실적 SBHI,수출전망 SBHI,영업이익 실적 SBHI,영업이익 전망 SBHI,자금사정 실적 SBHI,자금사정 전망 SBHI,고용수준 실적 SBHI,고용수준 전망 SBHI
0,2018Q1,서비스업,87.7,89.2,87.9,88.9,88,88.5,82.7,84.7,79,82.6,93.5,95.1
1,2018Q1,도매 및 소매업,89.9,87.8,89.6,87.5,88,88.5,83.3,82.8,78.3,81.8,92.4,95.3
2,2018Q1,수리 및 기타개인서비스업,73.5,92.6,74.2,93.1,73,109.6,72.4,86.9,67.1,82.1,93,90.9
3,2018Q1,운수업,83.7,91.2,83.9,92.9,111.8,79.7,81.4,92,79.1,87.1,89.3,89.8
4,2018Q1,숙박 및 음식점업,73.3,89.3,78.7,87.9,-,-,72,79.8,71.3,76.7,103.9,101.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,2022Q4,부동산업 및 임대업,70.1,67.4,70.1,68.1,72.9,100,68.5,65.4,70.9,68.1,100.2,94.6
216,2022Q4,전문과학 및 기술서비스업,85.9,87.8,86.3,87.2,75.7,74.8,83.4,86.8,84.9,86.4,94.4,93.3
217,2022Q4,사업시설관리 및 사업지원서비스업,85.8,88.9,85.3,88,97.2,89,90.3,86.2,90.2,85.2,83.9,83.8
218,2022Q4,교육서비스업,76.7,87.2,76.6,86.5,-,-,75.2,86.4,75.9,83.4,99.2,99.2


In [266]:
# 중소기업 경기동행종합지수

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = temp_df['C1_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        # 'DT'컬럼의 이름을 대체함
        newColName = f"{temp_df['TBL_NM'].unique()[0][:]}_{s}"        
        filtered = temp_df[temp_df['C1_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
        split_dfs[newColName]['PRD_DE'] = split_dfs[newColName]['PRD_DE'].apply(month_to_quarter)
        # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
        split_dfs[newColName] = split_dfs[newColName][~split_dfs[newColName]['PRD_DE'].str.contains('Unknown')]
    return split_dfs

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df.iloc[:, [2, 4]].copy()
    for name, sub in list(refDf.items())[1:]:
        sub_unique = sub.drop_duplicates(subset=['PRD_DE'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 4]], on=['PRD_DE'], how='left')  
    return df_merged
    
def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return df.merge(refDf, on='PRD_DE', how='left')  


##################### 중소기업 경기동행종합지수 ##################### 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=303&tblId=DT_303005_CI001'
coincidentIndex = mergeTimeSeriesDataFrame(mergeDataFrame(dataRefine(urlRequest(url))))
display(coincidentIndex)
save_excel_and_csv(coincidentIndex, 'coincidentIndex')
################################################################# 

,PRD_DE,중소기업 경기동행종합지수_동행종합지수,중소기업 경기동행종합지수_동행지수 순환변동치,중소기업 경기동행종합지수_동행지수 전월비,중소기업 경기동행종합지수_동행지수 전월차,중소기업 경기동행종합지수_동행지수 전년동기대비,중소기업 경기동행종합지수_순환변동치 전월비,중소기업 경기동행종합지수_순환변동치 전월차,중소기업 경기동행종합지수_순환변동치 전년동기대비
0,2018Q1,102.43,100.78,0.13,0.13,0.14,0.11,0.11,-0.14
1,2018Q2,102.39,100.69,0.01,0.01,0.06,-0.01,-0.01,-0.18
2,2018Q3,102.13,100.4,-0.07,-0.07,-0.5,-0.08,-0.08,-0.69
3,2018Q4,102.18,100.42,-0.07,-0.07,0.2,-0.07,-0.07,0.03
4,2019Q1,102.39,100.6,0.22,0.22,-0.04,0.22,0.22,-0.18
5,2019Q2,102.47,100.65,-0.12,-0.12,0.08,-0.13,-0.13,-0.04
6,2019Q3,102.26,100.42,-0.17,-0.17,0.13,-0.17,-0.17,0.02
7,2019Q4,102.48,100.6,0.36,0.37,0.29,0.36,0.36,0.18
8,2020Q1,100.47,98.58,-1.27,-1.29,-1.88,-1.28,-1.28,-2.01
9,2020Q2,97.56,95.68,-0.24,-0.23,-4.79,-0.24,-0.23,-4.94


In [267]:
# 산업별 기업규모별 기업수(중소기업-소상공인)
# 산업별 기업규모별 종사자수(중소기업-소상공인)

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = temp_df['ITM_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        # 'DT'컬럼의 이름을 대체함
        newColName = f"{temp_df['TBL_NM'].unique()[0][:]}_{s}"        
        filtered = temp_df[temp_df['ITM_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
    return split_dfs

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df.iloc[:, [2, 3, 4]].copy()
    for name, sub in list(refDf.items())[1:]:
        sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4]], on=['PRD_DE', 'C1_NM'], how='left')  
    return df_merged


##################### 산업별 기업규모별 기업수(중소기업-소상공인) #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+&objL1=G+H+I+L+N+P+Q+R+S+&objL2=2+3+4+5+6+7+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=DT_1BD1022'
induCorpCount = mergeDataFrame(dataRefine(urlRequest(url)))
display(induCorpCount)
save_excel_and_csv(induCorpCount, 'induCorpCount')
####################################################################################


###################### 산업별 기업규모별 종사자수(중소기업-소상공인) ################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+&objL1=G+H+I+L+N+P+Q+R+S+&objL2=5+6+7+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=DT_1BD1023'
induEmployeeCount = mergeDataFrame(dataRefine(urlRequest(url)))
display(induEmployeeCount)
save_excel_and_csv(induEmployeeCount, 'induEmployeeCount')
####################################################################################

,PRD_DE,C1_NM,산업별 기업규모별 기업수_활동,산업별 기업규모별 기업수_신생,산업별 기업규모별 기업수_소멸
0,2018,도매 및 소매업,193,4,-
1,2019,도매 및 소매업,192,2,-
2,2020,도매 및 소매업,200,6,1
3,2021,도매 및 소매업,212,10,2
4,2022,도매 및 소매업,198,15,1
...,...,...,...,...,...
265,2018,협회 및 단체 수리 및 기타 개인서비스업,275498,-,-
266,2019,협회 및 단체 수리 및 기타 개인서비스업,295356,2,-
267,2020,협회 및 단체 수리 및 기타 개인서비스업,308196,1,-
268,2021,협회 및 단체 수리 및 기타 개인서비스업,318346,1,0


,PRD_DE,C1_NM,산업별 기업규모별 종사자수_활동,산업별 기업규모별 종사자수_신생,산업별 기업규모별 종사자수_소멸
0,2018,도매 및 소매업,3061,262,217
1,2019,도매 및 소매업,3119,274,212
2,2020,도매 및 소매업,3216,295,217
3,2021,도매 및 소매업,3317,300,216
4,2022,도매 및 소매업,3376,285,228
...,...,...,...,...,...
130,2018,협회 및 단체 수리 및 기타 개인서비스업,324,58,40
131,2019,협회 및 단체 수리 및 기타 개인서비스업,346,64,44
132,2020,협회 및 단체 수리 및 기타 개인서비스업,367,57,44
133,2021,협회 및 단체 수리 및 기타 개인서비스업,372,58,41


In [268]:
# 시도/산업중분류별/조직형태별 사업체 및 종사자수
# 시도/산업중분류별/사업장점유형태별 사업체수

# dataRefine 함수
def dataRefine(refDf, text):
    if text == 'pre2020':
        tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'C2_NM', 'C3_NM', 'DT']].copy()
        filter_1 = tempDf['C3_NM'].drop_duplicates().values.tolist()
        filter_2 = tempDf['ITM_NM'].drop_duplicates().values.tolist()
        split_dfs = {}    
        for s in filter_1:
            for p in filter_2:
                # 'DT'컬럼의 이름을 대체함
                newColName = f"{tempDf['TBL_NM'].unique()[0][:]}_{p}_{s}"  
                filtered = tempDf[(tempDf['C3_NM'] == s) & (tempDf['ITM_NM'] == p)]
                split_dfs[newColName] = filtered.copy()
                split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
    else:
        tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'C2_NM', 'DT']].copy()
        filter_1 = tempDf['ITM_NM'].drop_duplicates().values.tolist()
        split_dfs = {}
        for s in filter_1:
            newColName = f"{tempDf['TBL_NM'].unique()[0][:]}_{s}"     
            filtered = tempDf[tempDf['ITM_NM'] == s]
            split_dfs[newColName] = filtered.copy()
            split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
    return split_dfs
 
def mergeDataFrame(refDf, text):
    first_df = list(refDf.values())[0]
    if text == 'pre2020':
        df_merged = first_df.iloc[:, [2, 3, 4, 6]].copy()
        for name, sub in list(refDf.items())[1:]:
            sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM', 'C2_NM'])
            df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4, 6]], on=['PRD_DE', 'C1_NM', 'C2_NM'], how='left')
    else:
        df_merged = first_df.iloc[:, [2, 3, 4, 5]].copy()
        for name, sub in list(refDf.items())[1:]:
            sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM', 'C2_NM'])
            df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4, 5]], on=['PRD_DE', 'C1_NM', 'C2_NM'], how='left')
    return df_merged



########## 시도/산업중분류별/조직형태별 사업체 및 종사자수 ##########
# 2020년 이전
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=ALL&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe=2019&orgId=142&tblId=DT_1ME0101' 
localInduCorpEmployeeCount_pre2020 = mergeDataFrame(dataRefine(urlRequest(url), 'pre2020'), 'pre2020')
display(localInduCorpEmployeeCount_pre2020)

# 2020년 이후
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+T04+T05+T06+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe={endYYYY}&orgId=142&tblId=DT_2ME0101'
localInduCorpEmployeeCount_post2020 = mergeDataFrame(dataRefine(urlRequest(url), 'post2020'), 'post2020')
display(localInduCorpEmployeeCount_post2020)

# 두 개의 dataframe을 합침
# 컬럼명이 다르므로 'localInduCorpEmployeeCount_post2020' 컬럼명으로 통일 시키고 concat로 합침
localInduCorpEmployeeCount_pre2020.columns = localInduCorpEmployeeCount_post2020.columns
localInduCorpEmployeeCount = pd.concat([localInduCorpEmployeeCount_pre2020, localInduCorpEmployeeCount_post2020], ignore_index=True)
display(localInduCorpEmployeeCount)
save_excel_and_csv(localInduCorpEmployeeCount, 'localInduCorpEmployeeCount')
#########################################################################


########## 시도/산업중분류별/사업장점유형태별 사업체수 ##########
# 2020년 이전
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=ALL&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe=2019&orgId=142&tblId=DT_1ME0303'
localInduCorpCount_pre2020 = mergeDataFrame(dataRefine(urlRequest(url), 'pre2020'), 'pre2020')

# 2020년 이후
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe={endYYYY}&orgId=142&tblId=DT_2ME0303'
localInduCorpCount_post2020 = mergeDataFrame(dataRefine(urlRequest(url), 'post2020'), 'post2020')

# 두 개의 dataframe을 합침
# 컬럼명이 다르므로 'localInduCorpCount_post2020' 컬럼명으로 통일 시키고 concat로 합침
localInduCorpCount_pre2020.columns = localInduCorpCount_post2020.columns
localInduCorpCount = pd.concat([localInduCorpCount_pre2020, localInduCorpCount_post2020], ignore_index=True)
display(localInduCorpCount)
save_excel_and_csv(localInduCorpCount, 'localInduCorpCount')
#########################################################################

,PRD_DE,C1_NM,C2_NM,시도/산업중분류별/조직형태별 사업체 및 종사자수_사업체수_합계,시도/산업중분류별/조직형태별 사업체 및 종사자수_종사자수_합계,시도/산업중분류별/조직형태별 사업체 및 종사자수_사업체수_개인,시도/산업중분류별/조직형태별 사업체 및 종사자수_종사자수_개인,시도/산업중분류별/조직형태별 사업체 및 종사자수_사업체수_법인,시도/산업중분류별/조직형태별 사업체 및 종사자수_종사자수_법인
0,2018,전국,도매 및 소매업,890665,1827061,813944,1622282,76721,204779
1,2019,전국,도매 및 소매업,891257,1870109,803402,1634602,87855,235507
2,2018,전국,자동차 및 부품 판매업,26056,69912,23060,58805,2996,11107
3,2019,전국,자동차 및 부품 판매업,26024,68841,22556,56710,3468,12131
4,2018,전국,도매 및 상품중개업,258452,544189,201701,395386,56751,148803
...,...,...,...,...,...,...,...,...,...
275,2019,제주특별자치도,교육 서비스업,2000,4108,2000,4108,0,0
276,2018,제주특별자치도,예술 스포츠 및 여가관련 서비스업,1249,2431,1228,2361,21,70
277,2019,제주특별자치도,예술 스포츠 및 여가관련 서비스업,1328,2684,1325,2679,3,5
278,2018,제주특별자치도,수리 및 기타 개인서비스업,3961,5744,3917,5655,44,89


,PRD_DE,C1_NM,C2_NM,시도/산업중분류별/조직형태별 사업체 및 종사자수_총 사업체 수,시도/산업중분류별/조직형태별 사업체 및 종사자수_총 종사자 수,시도/산업중분류별/조직형태별 사업체 및 종사자수_개인_사업체 수,시도/산업중분류별/조직형태별 사업체 및 종사자수_개인_종사자 수,시도/산업중분류별/조직형태별 사업체 및 종사자수_법인_사업체 수,시도/산업중분류별/조직형태별 사업체 및 종사자수_법인_종사자 수
0,2020,전국,도매 및 소매업,1384830,2150571,1246345,1877568,138485,273003
1,2021,전국,도매 및 소매업,1362910,2117974,1216253,1849892,146657,268082
2,2022,전국,도매 및 소매업,1350842,2057241,1209465,1794727,141377,262514
3,2020,전국,자동차 및 부품 판매업,34998,71895,30289,60660,4709,11235
4,2021,전국,자동차 및 부품 판매업,34445,70453,29884,58087,4561,12366
...,...,...,...,...,...,...,...,...,...
415,2021,제주특별자치도,예술 스포츠 및 여가관련 서비스업,1535,2527,1480,2393,55,134
416,2022,제주특별자치도,예술 스포츠 및 여가관련 서비스업,1565,2599,1508,2530,57,69
417,2020,제주특별자치도,협회 및 단체 수리 및 기타 개인서비스업,4977,6393,4961,6317,16,76
418,2021,제주특별자치도,협회 및 단체 수리 및 기타 개인서비스업,5042,6605,4992,6261,50,344


,PRD_DE,C1_NM,C2_NM,시도/산업중분류별/조직형태별 사업체 및 종사자수_총 사업체 수,시도/산업중분류별/조직형태별 사업체 및 종사자수_총 종사자 수,시도/산업중분류별/조직형태별 사업체 및 종사자수_개인_사업체 수,시도/산업중분류별/조직형태별 사업체 및 종사자수_개인_종사자 수,시도/산업중분류별/조직형태별 사업체 및 종사자수_법인_사업체 수,시도/산업중분류별/조직형태별 사업체 및 종사자수_법인_종사자 수
0,2018,전국,도매 및 소매업,890665,1827061,813944,1622282,76721,204779
1,2019,전국,도매 및 소매업,891257,1870109,803402,1634602,87855,235507
2,2018,전국,자동차 및 부품 판매업,26056,69912,23060,58805,2996,11107
3,2019,전국,자동차 및 부품 판매업,26024,68841,22556,56710,3468,12131
4,2018,전국,도매 및 상품중개업,258452,544189,201701,395386,56751,148803
...,...,...,...,...,...,...,...,...,...
695,2021,제주특별자치도,예술 스포츠 및 여가관련 서비스업,1535,2527,1480,2393,55,134
696,2022,제주특별자치도,예술 스포츠 및 여가관련 서비스업,1565,2599,1508,2530,57,69
697,2020,제주특별자치도,협회 및 단체 수리 및 기타 개인서비스업,4977,6393,4961,6317,16,76
698,2021,제주특별자치도,협회 및 단체 수리 및 기타 개인서비스업,5042,6605,4992,6261,50,344


,PRD_DE,C1_NM,C2_NM,시도/산업중분류별/사업장점유형태별 사업체수_사업체 수,시도/산업중분류별/사업장점유형태별 사업체수_소유,시도/산업중분류별/사업장점유형태별 사업체수_임차(전대차 포함)
0,2018,전국,도매 및 소매업,890665,189188,693270
1,2019,전국,도매 및 소매업,891257,182091,709166
2,2018,전국,자동차 및 부품 판매업,26056,5930,19797
3,2019,전국,자동차 및 부품 판매업,26024,5735,20289
4,2018,전국,도매 및 상품중개업,258452,58729,197708
...,...,...,...,...,...,...
100,2021,전국,개인 및 소비용품 수리업,96434,32637,63797
101,2022,전국,개인 및 소비용품 수리업,93799,31598,62201
102,2020,전국,기타 개인 서비스업,245648,35441,210207
103,2021,전국,기타 개인 서비스업,244686,43543,201143


In [269]:
# 시도/산업중분류별/사업체당 보증금 및 월세, 매출대비 지급비율(단위: 만원, %)
# 시도/산업중분류별 영업비용
# 시도/산업중분류별/영업이익규모별 사업체수 비율(개, 백만원, %)
# 시도별 경제활동별 지역내총생산(백만원, %p)
# 시도별 산업중분류별 기업규모별 매출액(소상공인, 억원)

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'C2_NM', 'DT']].copy()
    filter_1 = tempDf['ITM_NM'].drop_duplicates().values.tolist()
    split_dfs = {}
    for s in filter_1:
        newColName = f"{tempDf['TBL_NM'].unique()[0][:]}_{s}"     
        filtered = tempDf[tempDf['ITM_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
    return split_dfs 

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df.iloc[:, [2, 3, 4, 5]].copy()
    for name, sub in list(refDf.items())[1:]:
        sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM', 'C2_NM'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4, 5]], on=['PRD_DE', 'C1_NM', 'C2_NM'], how='left')
    return df_merged



########## 시도/산업중분류별/사업체당 보증금 및 월세, 매출대비 지급비율(단위: 만원, %) ##########
# 2020년 이전
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+T04+T05+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe=2019&orgId=142&tblId=DT_1ME0305'
df = urlRequest(url)
refinedDf = dataRefine(df)
localInduDepositCost_pre2020 = mergeDataFrame(refinedDf)

# 2020년 이후
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+T04+T05+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe={endYYYY}&orgId=142&tblId=DT_2ME0305'
df = urlRequest(url)
refinedDf = dataRefine(df)
localInduDepositCost_post2020 = mergeDataFrame(refinedDf)

# 두 개의 dataframe을 합침
# 컬럼명이 다르므로 'localInduCorpCount_post2020' 컬럼명으로 통일 시키고 concat로 합침
localInduDepositCost_pre2020.columns = localInduDepositCost_post2020.columns
localInduDepositCost = pd.concat([localInduDepositCost_pre2020, localInduDepositCost_post2020], ignore_index=True)
display(localInduDepositCost)
save_excel_and_csv(localInduDepositCost, 'localInduDepositCost')
##########################################################################################


########## 시도/산업중분류별 영업비용 ####################
# 2020년 이전
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=00+10+50+20+30+40+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe=2019&orgId=142&tblId=DT_1ME0311'
localInduOpCost_pre2020 = mergeDataFrame(dataRefine(urlRequest(url)))

# 2020년 이후
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T06+T03+T04+T05+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe={endYYYY}&orgId=142&tblId=DT_2ME0311'
localInduOpCost_post2020 = mergeDataFrame(dataRefine(urlRequest(url)))

# 두 개의 dataframe을 합침
# 컬럼명이 다르므로 'localInduCorpCount_post2020' 컬럼명으로 통일 시키고 concat로 합침
localInduOpCost_pre2020.columns = localInduOpCost_post2020.columns
localInduOpCost = pd.concat([localInduOpCost_pre2020, localInduOpCost_post2020], ignore_index=True)
display(localInduOpCost)
save_excel_and_csv(localInduOpCost, 'localInduOpCost')
##########################################################################################


########## 시도/산업중분류별/영업이익규모별 사업체수 비율(개, 백만원, %) ####################
# 2020년 이전
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=00+10+20+30+40+50+60+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2018&endPrdDe=2020&orgId=142&tblId=DT_1ME0312'
localInduOpprofitCorpCount_pre2020 = mergeDataFrame(dataRefine(urlRequest(url)))

# 2020년 이후
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+T04+T05+T06+T07+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe=2022&orgId=142&tblId=DT_2ME0312'
localInduOpprofitCorpCount_post2020 = mergeDataFrame(dataRefine(urlRequest(url)))

# 두 개의 dataframe을 합침
# 컬럼명이 다르므로 'localInduOpprofitCorpCount_post2020' 컬럼명으로 통일 시키고 concat로 합침
localInduOpprofitCorpCount_pre2020.columns = localInduOpprofitCorpCount_post2020.columns
localInduOpprofitCorpCount = pd.concat([localInduOpprofitCorpCount_pre2020, localInduOpprofitCorpCount_post2020], ignore_index=True)
display(localInduOpprofitCorpCount)
save_excel_and_csv(localInduOpprofitCorpCount, 'localInduOpprofitCorpCount')
##########################################################################################


#################### 시도별 경제활동별 지역내총생산(백만원, %p) #################### 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T1+T2+T3+&objL1=ALL&objL2=SER+G00+H00+I00+L00+N00+O00+P00+Q00+R00+S00+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=DT_1C91'
localInduGDP = mergeDataFrame(dataRefine(urlRequest(url)))
display(localInduGDP)
save_excel_and_csv(localInduGDP, 'localInduGDP')
##########################################################################################


#################### 시도별 산업중분류별 기업규모별 매출액(소상공인, 억원) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=IM_G+IM_H+IM_I+IM_L+IM_N+IM_P+IM_Q+IM_R+IM_S+&objL2=ALL&objL3=16142T2524+&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=142&tblId=DT_BR_C001'
localInduRevenue = mergeDataFrame(dataRefine(urlRequest(url)))
display(localInduRevenue)
save_excel_and_csv(localInduRevenue, 'localInduRevenue')
##########################################################################################


,PRD_DE,C1_NM,C2_NM,시도/산업중분류별/사업체당 보증금 및 월세 매출대비 지급비율_보증금 있는 월세_보증금(만원),시도/산업중분류별/사업체당 보증금 및 월세 매출대비 지급비율_보증금 있는 월세_월세(만원),시도/산업중분류별/사업체당 보증금 및 월세 매출대비 지급비율_보증금 없는 월세(만원),시도/산업중분류별/사업체당 보증금 및 월세 매출대비 지급비율_전세(만원),시도/산업중분류별/사업체당 보증금 및 월세 매출대비 지급비율_매출대비 일정비율 지급(%)
0,2018,전국,도매 및 소매업,2482,132,70,4338,21.2
1,2019,전국,도매 및 소매업,2651,138,79,4504,18.7
2,2018,전국,자동차 및 부품 판매업,3272,190,110,2158,30
3,2019,전국,자동차 및 부품 판매업,3279,173,165,3597,30
4,2018,전국,도매 및 상품중개업,1865,122,59,5980,10.4
...,...,...,...,...,...,...,...,...
100,2021,전국,개인 및 소비용품 수리업,1753,106,51,4166,12.6
101,2022,전국,개인 및 소비용품 수리업,1958,120,59,3241,15.4
102,2020,전국,기타 개인 서비스업,1875,90,60,2445,11.6
103,2021,전국,기타 개인 서비스업,1799,91,60,4283,0


,PRD_DE,C1_NM,C2_NM,시도/산업중분류별/영업비용_사업체 수,시도/산업중분류별/영업비용_영업비용,시도/산업중분류별/영업비용_영업비용_급여총액,시도/산업중분류별/영업비용_영업비용_임차료,시도/산업중분류별/영업비용_영업비용_기타,시도/산업중분류별/영업비용_영업비용_매출원가
0,2018,전국,도매 및 소매업,890665,265231259,21055921,13417319,230758018,NaN
1,2019,전국,도매 및 소매업,891257,271182966,21953961,13885283,21294860,214048861
2,2018,전국,자동차 및 부품 판매업,26056,12621841,1267765,655119,10698958,NaN
3,2019,전국,자동차 및 부품 판매업,26024,13202215,1153301,556963,1444886,10047065
4,2018,전국,도매 및 상품중개업,258452,141033251,10440229,3513624,127079398,NaN
...,...,...,...,...,...,...,...,...,...
1120,2021,제주특별자치도,예술 스포츠 및 여가관련 서비스업,1535,78733,15274,19041,28026,16393
1121,2022,제주특별자치도,예술 스포츠 및 여가관련 서비스업,1565,85414,18708,20928,26918,18860
1122,2020,제주특별자치도,협회 및 단체 수리 및 기타 개인서비스업,4977,186896,29416,29876,62894,64710
1123,2021,제주특별자치도,협회 및 단체 수리 및 기타 개인서비스업,5042,159679,18647,21738,41416,77878


,PRD_DE,C1_NM,C2_NM,시도/산업중분류별/영업이익규모별 사업체수 비율_사업체 수,시도/산업중분류별/영업이익규모별 사업체수 비율_영업이익,시도/산업중분류별/영업이익규모별 사업체수 비율_비율합계(%),시도/산업중분류별/영업이익규모별 사업체수 비율_1천만원미만(%),시도/산업중분류별/영업이익규모별 사업체수 비율_1천만원~3천만원미만(%),시도/산업중분류별/영업이익규모별 사업체수 비율_3천만원~5천만원미만(%),시도/산업중분류별/영업이익규모별 사업체수 비율_5천만원 이상(%)
0,2018,전국,도매 및 소매업,890665,36218057,100,16.9,32.7,24.1,26.3
1,2019,전국,도매 및 소매업,891257,35245359,100,18.9,32.2,21.8,27.1
2,2020,전국,도매 및 소매업,911339,18964436,100,45.7,31.4,10.8,12.1
3,2018,전국,자동차 및 부품 판매업,26056,1589727,100,11.5,25.3,29,34.2
4,2019,전국,자동차 및 부품 판매업,26024,1397269,100,15.6,26.6,23,34.8
...,...,...,...,...,...,...,...,...,...,...
1345,2021,제주특별자치도,예술 스포츠 및 여가관련 서비스업,1535,17032,100,44.8,37.1,11.1,7
1346,2022,제주특별자치도,예술 스포츠 및 여가관련 서비스업,1565,27189,100,37.6,34.1,19.7,8.6
1347,2020,제주특별자치도,협회 및 단체 수리 및 기타 개인서비스업,4977,38309,100,45.7,49,5,0.3
1348,2021,제주특별자치도,협회 및 단체 수리 및 기타 개인서비스업,5042,69484,100,31.3,53.1,7.5,8.1


,PRD_DE,C1_NM,C2_NM,시도별 경제활동별 지역내총생산_명목,시도별 경제활동별 지역내총생산_실질,시도별 경제활동별 지역내총생산_실질기여도
0,2018,전국,도매 및 소매업,175881303,169194046,0.24
1,2019,전국,도매 및 소매업,178189719,175673015,0.34
2,2020,전국,도매 및 소매업,175029822,175029822,-0.03
3,2021,전국,도매 및 소매업,185232614,177655406,0.13
4,2022,전국,도매 및 소매업,196172756,184163356,0.31
...,...,...,...,...,...,...
985,2018,제주특별자치도,서비스업,15501962,15748177,3.95
986,2019,제주특별자치도,서비스업,16125739,16292940,2.43
987,2020,제주특별자치도,서비스업,15498648,15498648,-3.53
988,2021,제주특별자치도,서비스업,16631695,16279688,3.6


,PRD_DE,C1_NM,C2_NM,시도별·산업중분류별·기업규모별 매출액_매출액
0,2019,G.도매 및 소매업(45~47),전국,3932001
1,2020,G.도매 및 소매업(45~47),전국,4015885
2,2021,G.도매 및 소매업(45~47),전국,4400729
3,2022,G.도매 및 소매업(45~47),전국,4667061
4,2019,G.도매 및 소매업(45~47),서울,926650
...,...,...,...,...
643,2022,S.협회 및 단체 수리 및 기타 개인 서비스업(94~96),제주,2654
644,2019,S.협회 및 단체 수리 및 기타 개인 서비스업(94~96),세종,818
645,2020,S.협회 및 단체 수리 및 기타 개인 서비스업(94~96),세종,868
646,2021,S.협회 및 단체 수리 및 기타 개인 서비스업(94~96),세종,961


In [270]:
# 시도별 지역내총생산에 대한 지출
# 시도별 1인당 지역내총생산, 지역총소득, 개인소득

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    split_dfs = {}
    filter_1 = temp_df['ITM_NM'].drop_duplicates().values.tolist()
    for s in filter_1:
        # 'DT'컬럼의 이름을 대체함
        newColName = f"{temp_df['TBL_NM'].unique()[0][:]}_{s}"        
        filtered = temp_df[temp_df['ITM_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
    return split_dfs

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df.iloc[:, [2, 3, 4]].copy()
    for name, sub in list(refDf.items())[1:]:
        sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4]], on=['PRD_DE', 'C1_NM'], how='left')  
    return df_merged

#################### 시도별 지역내총생산에 대한 지출 ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+&objL1=ALL&objL2=E00+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=DT_1C93'
localInduGDPExpense = mergeDataFrame(dataRefine(urlRequest(url)))
display(localInduGDPExpense)
save_excel_and_csv(localInduGDPExpense, 'localInduGDPExpense')
#########################################################################


#################### 시도별 1인당 지역내총생산, 지역총소득, 개인소득 #################### 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T1+T2+T3+T4+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=DT_1C96'
localGDPperPerson = mergeDataFrame(dataRefine(urlRequest(url)))
display(localGDPperPerson)
save_excel_and_csv(localGDPperPerson, 'localGDPperPerson')
#########################################################################

,PRD_DE,C1_NM,시도별 지역내총생산에 대한 지출_명목,시도별 지역내총생산에 대한 지출_실질,시도별 지역내총생산에 대한 지출_실질기여도
0,2018,전국,2007811627,2026457077,3.0809064924
1,2019,전국,2042981250,2075340783,2.4122744559
2,2020,전국,2062489906,2062489906,-0.6192176751
3,2021,전국,2224179729,2155993359,4.5335229389
4,2022,전국,2327595674,2214910682,2.7327228644
...,...,...,...,...,...
85,2018,제주특별자치도,22040507,22777044,0.9435497133
86,2019,제주특별자치도,22267086,22951902,0.7676955889
87,2020,제주특별자치도,21668785,21668785,-5.5904620838
88,2021,제주특별자치도,22757094,22346390,3.127101958


,PRD_DE,C1_NM,시도별 1인당 지역내총생산 지역총소득 개인소득_1인당 지역내총생산,시도별 1인당 지역내총생산 지역총소득 개인소득_1인당 지역총소득,시도별 1인당 지역내총생산 지역총소득 개인소득_1인당 개인소득,시도별 1인당 지역내총생산 지역총소득 개인소득_1인당 민간소비
0,2018,전국,38922,39070,21288,19277
1,2019,전국,39467,39787,22289,19778
2,2020,전국,39789,40115,23042,19083
3,2021,전국,42963,43415,23726,20292
4,2022,전국,45045,45588,24971,22104
...,...,...,...,...,...,...
85,2018,제주특별자치도,33673,32497,19402,19144
86,2019,제주특별자치도,33545,32250,19979,19741
87,2020,제주특별자치도,32404,31825,20835,18988
88,2021,제주특별자치도,33861,36153,21396,19966


In [271]:
# 상권별 집합상가 임대가격지수(단위: 2024.2Q=100.0)
# 상권별 통합상가 임대가격지수(시계열이 짧음, 2024 1Q~)
# 상권별 소규모상가 임대가격지수(단위: 2024.2Q=100.0)
# 상권별 오피스 임대가격지수(단위: 2024.2Q=100.0)
# 상권별 중대형상가 임대가격지수(단위: 2024.2Q=100.0)


# 추출 데이터가 월단위 데이터이므로, 이를 Quater 단위로 변환
# 월 → 분기(Q1~Q4)로 변환하는 함수 : 3,6,9,12월 자료만 Quater로 변환하고 나머지는 Unknown 처리
# 원천데이터가 Q1~Q4가 아니라 '01~04'로 표기되어 있음
def month_to_quarter(prd):
    year = prd[:4]
    month = prd[-2:]
    if month in ['01']:
        quarter = 'Q1'
    elif month in ['02']:
        quarter = 'Q2'
    elif month in ['03']:
        quarter = 'Q3'
    elif month in ['04']:
        quarter = 'Q4'
    else:
        quarter = 'Unknown'
    return f"{year}{quarter}"

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    newColName = tempDf['TBL_NM'].unique()[0]          # 'DT' 컬럼 네임 교체
    tempDf.rename(columns={'DT': newColName}, inplace=True)
    tempDf['PRD_DE'] = tempDf['PRD_DE'].apply(month_to_quarter)
    return tempDf[~tempDf['PRD_DE'].str.contains('Unknown')]
    
def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return df.merge(refDf, on='PRD_DE', how='left') 



#################### 상권별 집합상가 임대가격지수(단위: 2024.2Q=100.0) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=A01+A02+A03+A04+A05+A06+A07+A08+A09+A10+A11+A12+A13+A14+A15+A16+A17+A18+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=408&tblId=DT_40801_N3201_06'
refinedDf = dataRefine(urlRequest(url))
collectiveShopDistrictRentIndex = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(collectiveShopDistrictRentIndex)
##########################################################################################


#################### 상권별 통합상가 임대가격지수(2024 1Q부터만 데이터가 있음) #################### 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202401&endPrdDe=202501&orgId=408&tblId=DT_40801_N5201_06'
refinedDf = dataRefine(urlRequest(url))
integratedShopDistrictRentIndex = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(integratedShopDistrictRentIndex)
##################################################################################################


#################### 상권별 소규모상가 임대가격지수(단위: 2024.2Q=100.0) #################### 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=A01+A02+A03+A04+A05+A06+A07+A08+A09+A10+A11+A12+A13+A14+A15+A16+A17+A18+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=408&tblId=DT_40801_N4201_06'
refinedDf = dataRefine(urlRequest(url))
smallShopDistrictRentIndex = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(smallShopDistrictRentIndex)
##########################################################################################


#################### 상권별 오피스 임대가격지수(단위: 2024.2Q=100.0) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=A01+A02+A03+A04+A05+A06+A07+A08+A10+A11+A12+A13+A14+A15+A16+A17+A18+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=408&tblId=DT_40801_N1201_06'
refinedDf = dataRefine(urlRequest(url))
officeShopDistrictRentIndex = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(officeShopDistrictRentIndex)
##########################################################################################


#################### 상권별 중대형상가 임대가격지수(단위: 2024.2Q=100.0) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=A01+A02+A03+A04+A05+A06+A07+A08+A09+A10+A11+A12+A13+A14+A15+A16+A17+A18+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=408&tblId=DT_40801_N2201_06'
refinedDf = dataRefine(urlRequest(url))
middleBigShopDistrictRentIndex = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(middleBigShopDistrictRentIndex)
##########################################################################################

# Dataframe을 결합하여 상권별 임대가격지수 Dataframe을 만듦 (상권별 통합상가 임대가격지수(2024 1Q부터만 데이터가 있음)는 결합대상 제외)
ShopDistrictRentIndex = collectiveShopDistrictRentIndex.merge(smallShopDistrictRentIndex, on=['PRD_DE', 'C1_NM'], how='left') \
               .merge(officeShopDistrictRentIndex, on=['PRD_DE', 'C1_NM'], how='left') \
               .merge(middleBigShopDistrictRentIndex, on=['PRD_DE', 'C1_NM'], how='left')
display(ShopDistrictRentIndex)
save_excel_and_csv(ShopDistrictRentIndex, 'ShopDistrictRentIndex')

,PRD_DE,C1_NM,상권별 집합 상가 임대가격지수
0,2018Q1,전국,104.9027925748
1,2018Q1,서울,101.6782698735
2,2018Q1,부산,106.5378163003
3,2018Q1,대구,107.6678524211
4,2018Q1,인천,103.6875732383
...,...,...,...
355,2022Q4,전북,102.2162019813
356,2022Q4,전남,101.926831868
357,2022Q4,경북,100.3270092161
358,2022Q4,경남,100.5155464566


,PRD_DE,C1_NM,상권별 통합 상가 임대가격지수
0,2018Q1,NaN,NaN
1,2018Q2,NaN,NaN
2,2018Q3,NaN,NaN
3,2018Q4,NaN,NaN
4,2019Q1,NaN,NaN
5,2019Q2,NaN,NaN
6,2019Q3,NaN,NaN
7,2019Q4,NaN,NaN
8,2020Q1,NaN,NaN
9,2020Q2,NaN,NaN


,PRD_DE,C1_NM,상권별 소규모 상가 임대가격지수
0,2018Q1,전국,106.4770839558
1,2018Q1,서울,101.631843564
2,2018Q1,부산,106.8450785492
3,2018Q1,대구,106.3837850553
4,2018Q1,인천,106.0021004897
...,...,...,...
355,2022Q4,전북,102.2948654595
356,2022Q4,전남,101.532991943
357,2022Q4,경북,100.6063540969
358,2022Q4,경남,101.6805429512


,PRD_DE,C1_NM,상권별 오피스 임대가격지수
0,2018Q1,전국,99.4575170344
1,2018Q1,서울,96.8386807594
2,2018Q1,부산,110.2405776648
3,2018Q1,대구,104.0552743584
4,2018Q1,인천,105.5112599134
...,...,...,...
335,2022Q4,전북,100.5416611915
336,2022Q4,전남,100.8299198982
337,2022Q4,경북,100.7586398884
338,2022Q4,경남,100.7185199024


,PRD_DE,C1_NM,상권별 중대형 상가 임대가격지수
0,2018Q1,전국,104.9779512231
1,2018Q1,서울,100.345987702
2,2018Q1,부산,106.6294403863
3,2018Q1,대구,106.3589619076
4,2018Q1,인천,104.2718987913
...,...,...,...
355,2022Q4,전북,102.0838261674
356,2022Q4,전남,100.9803470089
357,2022Q4,경북,100.7889957506
358,2022Q4,경남,102.1287583259


,PRD_DE,C1_NM,상권별 집합 상가 임대가격지수,상권별 소규모 상가 임대가격지수,상권별 오피스 임대가격지수,상권별 중대형 상가 임대가격지수
0,2018Q1,전국,104.9027925748,106.4770839558,99.4575170344,104.9779512231
1,2018Q1,서울,101.6782698735,101.631843564,96.8386807594,100.345987702
2,2018Q1,부산,106.5378163003,106.8450785492,110.2405776648,106.6294403863
3,2018Q1,대구,107.6678524211,106.3837850553,104.0552743584,106.3589619076
4,2018Q1,인천,103.6875732383,106.0021004897,105.5112599134,104.2718987913
...,...,...,...,...,...,...
355,2022Q4,전북,102.2162019813,102.2948654595,100.5416611915,102.0838261674
356,2022Q4,전남,101.926831868,101.532991943,100.8299198982,100.9803470089
357,2022Q4,경북,100.3270092161,100.6063540969,100.7586398884,100.7889957506
358,2022Q4,경남,100.5155464566,101.6805429512,100.7185199024,102.1287583259


In [272]:
# 상권별 소규모 상가 공실률(단위 : %)
# 상권별 오피스 공실률 (단위 : %)
# 상권별 중대형 상가 공실률(단위 : %)
# 상권별 집합 상가 공실률 (시계열이 22년 4분기 부터밖에 없음)

# 추출 데이터가 월단위 데이터이므로, 이를 Quater 단위로 변환
# 월 → 분기(Q1~Q4)로 변환하는 함수 : 3,6,9,12월 자료만 Quater로 변환하고 나머지는 Unknown 처리
# 원천데이터가 Q1~Q4가 아니라 '01~04'로 표기되어 있음
def month_to_quarter(prd):
    year = prd[:4]
    month = prd[-2:]
    if month in ['01']:
        quarter = 'Q1'
    elif month in ['02']:
        quarter = 'Q2'
    elif month in ['03']:
        quarter = 'Q3'
    elif month in ['04']:
        quarter = 'Q4'
    else:
        quarter = 'Unknown'
    return f"{year}{quarter}"

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    newColName = tempDf['TBL_NM'].unique()[0]          # 'DT' 컬럼 네임 교체
    tempDf.rename(columns={'DT': newColName}, inplace=True)
    tempDf['PRD_DE'] = tempDf['PRD_DE'].apply(month_to_quarter)
    return tempDf[~tempDf['PRD_DE'].str.contains('Unknown')]
    
def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return df.merge(refDf, on='PRD_DE', how='left') 


#################### 상권별 소규모 상가 공실률(단위 : %) ####################
# 2018
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201801&endPrdDe=201904&orgId=408&tblId=DT_40801_N420201_01'
refinedDf_2018 = dataRefine(urlRequest(url))

# 2019
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201901&endPrdDe=201904&orgId=408&tblId=DT_40801_N420201_02'
refinedDf_2019 = dataRefine(urlRequest(url))

# 2020
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202001&endPrdDe=202004&orgId=408&tblId=DT_40801_N420201_03'
refinedDf_2020 = dataRefine(urlRequest(url))

# 2021
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=600+611+626+627+628+629+630+631+636+641+642+643+644+645+646+647+648+650+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202101&endPrdDe=202104&orgId=408&tblId=DT_40801_N420201_04'
refinedDf_2021 = dataRefine(urlRequest(url))

# 2022
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=600+611+626+627+628+629+630+631+636+641+642+643+644+645+646+647+648+650+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202201&endPrdDe=202204&orgId=408&tblId=DT_40801_N420201_05'
refinedDf_2022 = dataRefine(urlRequest(url))

refinedDf = pd.concat([refinedDf_2018, refinedDf_2019, refinedDf_2020, refinedDf_2021, refinedDf_2022], ignore_index=True)
smallShopDistrictVacancyRate = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(smallShopDistrictVacancyRate)
################################################################################


#################### 상권별 오피스 공실률 (단위 : %) ####################
# 2018
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201801&endPrdDe=201804&orgId=408&tblId=DT_40801_N120201_01'
refinedDf_2018 = dataRefine(urlRequest(url))

# 2019
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201901&endPrdDe=201904&orgId=408&tblId=DT_40801_N120201_02'
refinedDf_2019 = dataRefine(urlRequest(url))

# 2020
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202001&endPrdDe=202004&orgId=408&tblId=DT_40801_N120201_03'
refinedDf_2020 = dataRefine(urlRequest(url))

# 2021
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=100+111+126+127+128+129+130+131+141+142+143+144+145+146+147+148+150+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202101&endPrdDe=202104&orgId=408&tblId=DT_40801_N120201_04'
refinedDf_2021 = dataRefine(urlRequest(url))

# 2022
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=100+111+126+127+128+129+130+131+141+142+143+144+145+146+147+148+150+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202201&endPrdDe=202204&orgId=408&tblId=DT_40801_N120201_05'
refinedDf_2022 = dataRefine(urlRequest(url))

refinedDf = pd.concat([refinedDf_2018, refinedDf_2019, refinedDf_2020, refinedDf_2021, refinedDf_2022], ignore_index=True)
officeShopDistrictVacancyRate = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(officeShopDistrictVacancyRate)
#########################################################################


#################### 상권별 중대형 상가 공실률(단위 : %) ####################
# 2018
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201801&endPrdDe=201804&orgId=408&tblId=DT_40801_N220201_01'
refinedDf_2018 = dataRefine(urlRequest(url))

# 2019
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201904&endPrdDe=201904&orgId=408&tblId=DT_40801_N220201_02'
refinedDf_2019 = dataRefine(urlRequest(url))

# 2020
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202001&endPrdDe=202004&orgId=408&tblId=DT_40801_N220201_03'
refinedDf_2020 = dataRefine(urlRequest(url))

# 2021
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=200+211+226+227+228+229+230+231+236+241+242+243+244+245+246+247+248+250+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202104&endPrdDe=202104&orgId=408&tblId=DT_40801_N220201_04'
refinedDf_2021 = dataRefine(urlRequest(url))

# 2022
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=200+211+226+227+228+229+230+231+236+241+242+243+244+245+246+247+248+250+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202201&endPrdDe=202204&orgId=408&tblId=DT_40801_N220201_05'
refinedDf_2022 = dataRefine(urlRequest(url))

refinedDf = pd.concat([refinedDf_2018, refinedDf_2019, refinedDf_2020, refinedDf_2021, refinedDf_2022], ignore_index=True)
middleBigShopDistrictVacancyRate = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
display(middleBigShopDistrictVacancyRate)
############################################################################

# Dataframe을 결합하여 상권별 임대가격지수 Dataframe을 만듦 (상권별 통합상가 임대가격지수(2024 1Q부터만 데이터가 있음)는 결합대상 제외)
shopDistrictVacancyRate = smallShopDistrictVacancyRate.merge(officeShopDistrictVacancyRate, on=['PRD_DE', 'C1_NM'], how='left') \
               .merge(middleBigShopDistrictVacancyRate, on=['PRD_DE', 'C1_NM'], how='left')
display(shopDistrictVacancyRate)
save_excel_and_csv(shopDistrictVacancyRate, 'shopDistrictVacancyRate')

,PRD_DE,C1_NM,상권별 소규모 상가 공실률
0,2018Q1,전국,4.7
1,2018Q1,서울,3.7
2,2018Q1,부산,6
3,2018Q1,대구,5.1
4,2018Q1,인천,4.4
...,...,...,...
355,2022Q4,전북,10.4968816714
356,2022Q4,전남,6.7621761604
357,2022Q4,경북,6.2488865808
358,2022Q4,경남,7.7068800103


,PRD_DE,C1_NM,상권별 오피스 공실률
0,2018Q1,전국,12.7
1,2018Q1,서울,11.9
2,2018Q1,부산,16
3,2018Q1,대구,11.1
4,2018Q1,인천,13.4
...,...,...,...
335,2022Q4,전북,15.2539751808
336,2022Q4,전남,24.1158573621
337,2022Q4,경북,22.0877965742
338,2022Q4,경남,17.2263960105


,PRD_DE,C1_NM,상권별 중대형 상가 공실률
0,2018Q1,전국,10.4
1,2018Q1,서울,7.7
2,2018Q1,부산,9.8
3,2018Q1,대구,12.5
4,2018Q1,인천,12.8
...,...,...,...
253,2022Q4,전북,17.7640310488
254,2022Q4,전남,11.9684232042
255,2022Q4,경북,19.893386352
256,2022Q4,경남,14.4089017424


,PRD_DE,C1_NM,상권별 소규모 상가 공실률,상권별 오피스 공실률,상권별 중대형 상가 공실률
0,2018Q1,전국,4.7,12.7,10.4
1,2018Q1,서울,3.7,11.9,7.7
2,2018Q1,부산,6,16,9.8
3,2018Q1,대구,5.1,11.1,12.5
4,2018Q1,인천,4.4,13.4,12.8
...,...,...,...,...,...
355,2022Q4,전북,10.4968816714,15.2539751808,17.7640310488
356,2022Q4,전남,6.7621761604,24.1158573621,11.9684232042
357,2022Q4,경북,6.2488865808,22.0877965742,19.893386352
358,2022Q4,경남,7.7068800103,17.2263960105,14.4089017424
